<a href="https://colab.research.google.com/github/bols-blue/google_colab/blob/main/wisper_auto_transcription_%E8%A9%B1%E8%80%85%E5%88%86%E9%9B%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

とりあえずこれで動く

In [ ]:
! apt-get install -y librosa-dev chex
!python -V
!pip install git+https://github.com/openai/whisper.git
!pip install -qq https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip
!pip install -qq ipython==7.34.0

!pip install git+https://github.com/yinruiqing/pyannote-whisper


google driveを追加して読み込めるように

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ファイルの確認用

In [ ]:
!ls "/content/drive/MyDrive/Meet Recordings"

In [ ]:
! ffmpeg -i /content/drive/file.mp4 -vn  -ac 1 ./output.wav

In [ ]:
import whisper
from pyannote.audio import Pipeline
from pyannote.audio import Audio
from pyannote_whisper.utils import diarize_text

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization",
                                    use_auth_token="your_token")
model = whisper.load_model("large")
diarization_result = pipeline("/content/output.wav")

from pyannote.audio import Audio
audio = Audio(sample_rate=16000, mono=True)
audio_file = "/content/output.wav"
write_text = ""
for segment, _, speaker in diarization_result.itertracks(yield_label=True):
    waveform, sample_rate = audio.crop(audio_file, segment)
    text = model.transcribe(waveform.squeeze().numpy(), language="ja")["text"]
    print(f"{segment.start:.2f}s {segment.end:.2f}s {speaker}: {text}")
    write_text += f"{segment.start:.2f}s {segment.end:.2f}s {speaker}: {text}\n"

with open(f"file.mp4.diarization.txt", "w") as f:
    f.write(write_text)

runtime.unassign()

# 新しいセクション